# Bab 14: Deep Computer Vision Menggunakan Convolutional Neural Networks (CNN)

### 1. Pendahuluan

Bab 14 beralih ke salah satu aplikasi *Deep Learning* yang paling sukses: **Computer Vision**. Selama dekade terakhir, *Deep Learning* telah merevolusi bidang ini, terutama berkat munculnya **Convolutional Neural Networks (CNNs)**. CNNs adalah arsitektur Jaringan Saraf Tiruan khusus yang dirancang untuk memproses data berstruktur *grid* seperti gambar.

Bab ini akan membahas:
* Arsitektur dasar CNN, termasuk lapisan konvolusional dan lapisan *pooling*.
* Cara membangun CNN untuk tugas klasifikasi gambar.
* Arsitektur CNN yang populer dan berkinerja tinggi.
* Teknik *transfer learning* untuk Computer Vision.
* Teknik *computer vision* lainnya seperti lokalisasi objek, deteksi objek, dan segmentasi.

---

### 2. Arsitektur Lapisan Konvolusional

Lapisan konvolusional adalah blok bangunan inti dari CNN. Lapisan ini menggunakan filter kecil (juga disebut *kernels*) yang "bergulir" di atas gambar input untuk menghasilkan *feature map*. Setiap neuron di *feature map* hanya terhubung ke sejumlah kecil neuron di lapisan sebelumnya, yang dikenal sebagai *receptive field*.

* **Padding:** Menambahkan nol di sekitar batas input untuk mempertahankan dimensi spasial. `padding="same"` akan menghasilkan output dengan ukuran yang sama dengan input. `padding="valid"` berarti tanpa padding.
* **Stride:** Jarak pergeseran filter di setiap langkah. Stride yang lebih besar akan menghasilkan *feature map* yang lebih kecil.

---

### 3. Lapisan Pooling

Lapisan *pooling* bertujuan untuk melakukan *subsampling* (mengecilkan ukuran) representasi input, yang membantu mengurangi beban komputasi, memori, dan jumlah parameter, sehingga mengurangi risiko *overfitting*.
* **Max Pooling:** Mengambil nilai maksimum dari setiap *region* input. Ini adalah jenis *pooling* yang paling umum.
* **Average Pooling:** Mengambil nilai rata-rata dari setiap *region*.

---

### 4. Arsitektur CNN

Arsitektur CNN yang umum biasanya terdiri dari tumpukan beberapa lapisan konvolusional, diikuti oleh fungsi aktivasi (misalnya, ReLU), lalu lapisan *pooling*, dan diulang beberapa kali. Menjelang akhir jaringan, beberapa lapisan *fully connected* (Dense) ditambahkan, dengan lapisan output akhir yang menggunakan fungsi aktivasi seperti softmax untuk klasifikasi.

#### Implementasi CNN dengan Keras untuk Klasifikasi Gambar

Berikut adalah contoh pembangunan CNN sederhana menggunakan Keras untuk dataset Fashion MNIST.


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Memuat dataset Fashion MNIST
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Normalisasi dan penyesuaian bentuk data
X_train_full = np.expand_dims(X_train_full, -1).astype(np.float32) / 255
X_test = np.expand_dims(X_test, -1).astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

# Membangun model CNN sederhana
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])

# Compile dan latih model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])
# history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
# score = model.evaluate(X_test, y_test)

c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


---

### 5. Menggunakan Model CNN Pre-trained (Transfer Learning)
Untuk sebagian besar masalah computer vision, lebih baik menggunakan model CNN yang sudah dilatih sebelumnya (pre-trained) pada dataset besar seperti ImageNet. Teknik ini disebut Transfer Learning.

Prosesnya:

1. Bekukan lapisan-lapisan awal (Freeze Layers): Bobot dari lapisan-lapisan konvolusional awal dari model pre-trained dibekukan (`layer.trainable = False`) karena mereka telah belajar mendeteksi fitur-fitur umum.
2. Ganti atau Tambah Lapisan Atas: Ganti lapisan output asli dengan lapisan baru yang sesuai dengan tugas Anda.
3. Latih Model: Latih model pada dataset baru Anda. Awalnya, hanya bobot dari lapisan baru yang akan diperbarui.
4. (Opsional) Fine-Tuning: Setelah beberapa epoch, Anda dapat "mencairkan" (unfreeze) beberapa lapisan atas dari model pre-trained dan melatih seluruh jaringan dengan learning rate yang sangat kecil.

In [2]:
# Contoh Transfer Learning dengan Xception
# (Memerlukan preprocessing gambar agar sesuai dengan input Xception)
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(10, activation="softmax")(avg) # Asumsikan 10 kelas
model_transfer = keras.Model(inputs=base_model.input, outputs=output)

# Bekukan lapisan-lapisan dari base model
for layer in base_model.layers:
    layer.trainable = False

# Compile dan latih
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model_transfer.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                       metrics=["accuracy"])
# history = model_transfer.fit(...)

c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(




### 6. Tugas Computer Vision Lainnya
Selain klasifikasi, CNN juga sangat kuat untuk tugas-tugas berikut:

### a. Klasifikasi dan Lokalisasi Objek
Tugas ini tidak hanya mengklasifikasikan objek dalam gambar tetapi juga memprediksi lokasinya melalui sebuah bounding box (kotak pembatas). Ini dilakukan dengan menambahkan output head kedua pada model CNN:

* **Head Klasifikasi**: Memprediksi kelas objek (menggunakan loss `categorical_crossentropy`).
* **Head Regresi**: Memprediksi 4 angka yang merepresentasikan koordinat dan ukuran bounding box (menggunakan loss `mean_squared_error`).

### b. Deteksi Objek (Object Detection)
Deteksi objek adalah tugas yang lebih menantang untuk mendeteksi dan melokalisasi banyak objek dalam satu gambar. Beberapa pendekatan populer:

* **Fully Convolutional Networks (FCN)**: Menggunakan CNN untuk menghasilkan feature map padat, di mana setiap sel di peta tersebut memprediksi apakah ada objek di sana beserta bounding box-nya.
* **You Only Look Once (YOLO)**: Model yang sangat cepat dan populer yang memprediksi semua bounding box dalam satu kali jalan (single pass).
* **Region-based CNN (R-CNN)** dan **turunannya (Fast R-CNN, Faster R-CNN)**: Mengusulkan beberapa region potensial di gambar lalu menjalankan pengklasifikasi pada setiap region.

### c. Segmentasi Semantik dan Instance
* **Segmentasi Semantik**: Mengklasifikasikan setiap piksel dalam gambar ke dalam kelas tertentu (misalnya, jalan, mobil, pejalan kaki, bangunan). Arsitektur yang umum digunakan adalah U-Net, yang memiliki jalur downsampling (encoder) dan upsampling (decoder).
* **Segmentasi Instance**: Mirip dengan segmentasi semantik, tetapi juga membedakan antar instance dari kelas yang sama (misalnya, mengidentifikasi setiap mobil secara terpisah).